<a href="https://colab.research.google.com/github/yazhini4225/Plant-Disease-Classification/blob/main/plant_diasease__classification_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow keras matplotlib numpy scikit-learn


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload kaggle.json


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"yazhini4225","key":"9915b6570f943903ee6094330618b493"}'}

In [ ]:
!ls

sample_data


In [ ]:
!mv "kaggle (1).json" kaggle.json

In [ ]:
!mkdir -p /root/.kaggle/
!mv kaggle.json /root/.kaggle/


In [ ]:
import os
os.chmod("/root/.kaggle/kaggle.json", 600)

In [ ]:
!kaggle datasets download -d emmarex/plantdisease

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 97% 641M/658M [00:07<00:00, 93.3MB/s]
100% 658M/658M [00:07<00:00, 91.7MB/s]


In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset


Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
100% 2.69G/2.70G [00:28<00:00, 201MB/s]
100% 2.70G/2.70G [00:28<00:00, 102MB/s]


In [ ]:
import zipfile

# Path to the downloaded dataset
zip_path = 'new-plant-diseases-dataset.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/plant_disease')


In [ ]:
import os

# Define the path to the extracted folder
dataset_dir = '/content/plant_disease'

# Check the contents of the dataset folder
for root, dirs, files in os.walk(dataset_dir):
    print(f'Folder: {root}')
    print(f'Files in {root}: {len(files)}')
    print(f'Folders in {root}: {dirs}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Define paths to your train and validation directories
train_dir = '/content/plant_disease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'
val_dir = '/content/plant_disease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid'

# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create train and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [ ]:
# Build the CNN model
model = models.Sequential()

# Add a convolutional layer, followed by max pooling
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Add more convolutional layers
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output and add a fully connected layer
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))

# Output layer with softmax activation for multi-class classification
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2196/2196 ━━━━━━━━━━━━━━━━━━━━ 456s 204ms/step - accuracy: 0.4376 - loss: 1.9796 - val_accuracy: 0.8140 - val_loss: 0.5836
Epoch 2/10
   1/2196 ━━━━━━━━━━━━━━━━━━━━ 1:18 36ms/step - accuracy: 0.8438 - loss: 0.6054

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2196/2196 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.8438 - loss: 0.6054 - val_accuracy: 1.0000 - val_loss: 0.1112
Epoch 3/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 436s 198ms/step - accuracy: 0.8018 - loss: 0.6204 - val_accuracy: 0.8122 - val_loss: 0.5793
Epoch 4/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 0s 13us/step - accuracy: 0.7500 - loss: 1.1535 - val_accuracy: 0.2500 - val_loss: 1.8849
Epoch 5/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 435s 197ms/step - accuracy: 0.8563 - loss: 0.4525 - val_accuracy: 0.9004 - val_loss: 0.3013
Epoch 6/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 0s 13us/step - accuracy: 0.9375 - loss: 0.2598 - val_accuracy: 1.0000 - val_loss: 0.1697
Epoch 7/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 442s 198ms/step - accuracy: 0.8857 - loss: 0.3594 - val_accuracy: 0.9223 - val_loss: 0.2370
Epoch 8/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 0s 9us/step - accuracy: 0.9375 - loss: 0.2320 - val_accuracy: 1.0000 - val_loss: 0.1164
Epoch 9/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 431s 196ms/step - accuracy: 0.9045 - loss: 0

In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

550/550 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.9338 - loss: 0.1952
Validation Loss: 0.19318446516990662
Validation Accuracy: 0.9354655146598816


In [ ]:
model.save('plant_disease_model.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mv plant_disease_model.h5 /content/drive/MyDrive/

In [ ]:
import os

# Specify the path to your dataset
dataset_path = '/content/plant_disease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'

# List all folders in the dataset
folders = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]

# Print the list of folders
print("Folders in the dataset:")
for folder in folders:
    print(folder)


Folders in the dataset:
Potato___healthy
Apple___Apple_scab
Tomato___Early_blight
Grape___Black_rot
Apple___Black_rot
Apple___healthy
Tomato___Target_Spot
Soybean___healthy
Apple___Cedar_apple_rust
Potato___Early_blight
Tomato___Leaf_Mold
Corn_(maize)___healthy
Tomato___Septoria_leaf_spot
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Northern_Leaf_Blight
Strawberry___Leaf_scorch
Tomato___Tomato_Yellow_Leaf_Curl_Virus
Strawberry___healthy
Grape___Esca_(Black_Measles)
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Pepper,_bell___Bacterial_spot
Tomato___Tomato_mosaic_virus
Tomato___Bacterial_spot
Tomato___Late_blight
Raspberry___healthy
Cherry_(including_sour)___healthy
Cherry_(including_sour)___Powdery_mildew
Orange___Haunglongbing_(Citrus_greening)
Grape___healthy
Tomato___Spider_mites Two-spotted_spider_mite
Squash___Powdery_mildew
Tomato___healthy
Peach___healthy
Pepper,_bell___healthy
Corn_(maize)___Common_rust_
Potato___Late_blight
Blueberry___healthy
Peach___Bacteri